*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.crm_cust_info LIMIT 5;

In [0]:
df=spark.table("bara_slaes_project.bronze.crm_cust_info")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'cst_id':'Customer_ID',
    'cst_key':'Customer_Key',
    'cst_firstname':'Customer_first_name',
    'cst_lastname':'Customer_last_name',
    'cst_marital_status':'marital_status',
    'cst_gndr':'Gender',
    'cst_create_date':'Customer_create_date'
}

#df = df.withColumnsRenamed(rename_mapping)
for old_name,new_name in rename_mapping.items():
    df = df.withColumnRenamed(old_name, new_name)

In [0]:
## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

In [0]:
df= df.dropna(subset=["Customer_ID"])
df=df.fillna(value="Unknown",subset=["Customer_first_name","Customer_last_name","marital_status","Gender"])

In [0]:
df.filter(col("Customer_ID")=="29466").show()

In [0]:
# remove duplicates form the primary key (Customer_ID)
from pyspark.sql import functions as F
from pyspark.sql.window import Window
window_spec_last =Window.partitionBy('Customer_ID').orderBy(F.asc('Customer_create_date'))
df_ranked_last=df.withColumn('row_num', F.row_number().over(window_spec_last))
#df_ranked_last.filter(df_ranked_last.row_num != 1).display()#.drop("row_num")
df=df_ranked_last.filter(df_ranked_last.row_num == 1).drop("row_num")

In [0]:
#Replace Values
Gender_replacements = {
    "F": "Female",
    "M": "Male",
    "U": "Unknown"
}
marital_status_replacements = {
    "S": "Single",
    "M": "Married",
    "Unknown": "Unknown",
}
df=df.replace(Gender_replacements,subset=["Gender"])
df=df.replace(marital_status_replacements,subset=["marital_status"])

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite").saveAsTable("bara_slaes_project.silver.customer_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.customer_info;